In [1]:
from flask import Flask
from flask_cors import CORS, cross_origin
from draughts import DraughtsBoard, DraughtsPiece
from chess import ChessBoard, ChessPiece
from minimax import best_move
from flask import jsonify
from base import Player
import re
import json

In [2]:
board_types = {'draughts': DraughtsBoard, 'chess': ChessBoard}
piece_types = {'draughts': DraughtsPiece, 'chess': ChessPiece}
#T = board_types['draughts']


In [3]:
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

def board_to_json(board):
    pos = {f"S{square[0]}{square[1]}": piece.name for square, piece in board.position.items() }
    tomove = board.player_to_move.name
    winner = board.winner()
    if winner is not None:
        winner = winner.name
    ret = {'pos': pos, 'tomove': tomove, 'winner': winner}
    ret = jsonify(ret)
    return ret
    
    
@app.route('/<game>/initial_position')
@cross_origin()
def initial_position(game):
    board_type = board_types[game]
    b = board_type()
    return board_to_json(b)

@app.route('/<game>/pos/<position_json>')
@cross_origin()
def make_move_given_position(game, position_json):
    "given a position, find the best move and return the new board with that move played"
#     print("in make_move_given_position")
    position = json.loads(position_json)
    
    tomove = position['tomove']
    board_pos = position['pos']
#     print("tomove: ", tomove)
#     print("board_pos: ", board_pos)
    
    board_type = board_types[game]
    piece_type = piece_types[game]
    
    
    board_pos = {(int(sq[1]), int(sq[2])): piece_type[piece] for sq, piece in board_pos.items()}
    board = board_type(board_pos, Player[tomove])
    move = best_move(board, 2)
    new_board = board.make_move(move)
    
    ret = board_to_json(new_board)
#     print("leaving make_move_given_position")
    return(ret)

@app.route('/<game>/move/<pos_and_move_json>')
@cross_origin()
def play_move(game, pos_and_move_json):
    """
    Given a position and a move, play that move and return the new board.
    If the move is illegal, just return the original position.
    The move is in the format S13_S34
    """
#     print("in play_move")


    pos = json.loads(pos_and_move_json)
    tomove = pos['tomove']
    board_pos = pos['pos']
    move = pos['move']
    
    move = ((int(move[1]), int(move[2])), (int(move[5]), int(move[6])))
    print(move)

    board_type = board_types[game]
    piece_type = piece_types[game]

    board_pos = {(int(sq[1]), int(sq[2])): piece_type[piece] for sq, piece in board_pos.items()}
    board = board_type(board_pos, Player[tomove])
    
    legal_moves = board.legal_moves()
    
    if move in legal_moves:
        new_board = board.make_move(move)
    else:
        new_board = board
    
    ret = board_to_json(new_board)
#     print("leaving play_move")
    return ret



In [4]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Sep/2020 13:23:52] "GET /chess/initial_position HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2020 13:23:54] "GET /chess/move/%7B%22pos%22:%7B%22S70%22:%22BR%22,%22S71%22:%22BN%22,%22S72%22:%22BB%22,%22S73%22:%22BQ%22,%22S74%22:%22BK%22,%22S75%22:%22BB%22,%22S76%22:%22BN%22,%22S77%22:%22BR%22,%22S60%22:%22BP%22,%22S61%22:%22BP%22,%22S62%22:%22BP%22,%22S63%22:%22BP%22,%22S64%22:%22BP%22,%22S65%22:%22BP%22,%22S66%22:%22BP%22,%22S67%22:%22BP%22,%22S50%22:%22E%22,%22S51%22:%22E%22,%22S52%22:%22E%22,%22S53%22:%22E%22,%22S54%22:%22E%22,%22S55%22:%22E%22,%22S56%22:%22E%22,%22S57%22:%22E%22,%22S40%22:%22E%22,%22S41%22:%22E%22,%22S42%22:%22E%22,%22S43%22:%22E%22,%22S44%22:%22E%22,%22S45%22:%22E%22,%22S46%22:%22E%22,%22S47%22:%22E%22,%22S30%22:%22E%22,%22S31%22:%22E%22,%22S32%22:%22E%22,%22S33%22:%22E%22,%22S34%22:%22E%22,%22S35%22:%22E%22,%22S36%22:%22E%22,%22S37%22:%22E%22,%22S20%22:%22E%22,%22S21%22:%22E%22,%22S22%22:%22E%2

((1, 4), (3, 4))


127.0.0.1 - - [06/Sep/2020 13:24:13] "GET /chess/pos/%7B%22pos%22:%7B%22S70%22:%22BR%22,%22S71%22:%22BN%22,%22S72%22:%22BB%22,%22S73%22:%22BQ%22,%22S74%22:%22BK%22,%22S75%22:%22BB%22,%22S76%22:%22BN%22,%22S77%22:%22BR%22,%22S60%22:%22BP%22,%22S61%22:%22BP%22,%22S62%22:%22BP%22,%22S63%22:%22BP%22,%22S64%22:%22BP%22,%22S65%22:%22BP%22,%22S66%22:%22BP%22,%22S67%22:%22BP%22,%22S50%22:%22E%22,%22S51%22:%22E%22,%22S52%22:%22E%22,%22S53%22:%22E%22,%22S54%22:%22E%22,%22S55%22:%22E%22,%22S56%22:%22E%22,%22S57%22:%22E%22,%22S40%22:%22E%22,%22S41%22:%22E%22,%22S42%22:%22E%22,%22S43%22:%22E%22,%22S44%22:%22E%22,%22S45%22:%22E%22,%22S46%22:%22E%22,%22S47%22:%22E%22,%22S30%22:%22E%22,%22S31%22:%22E%22,%22S32%22:%22E%22,%22S33%22:%22E%22,%22S34%22:%22WP%22,%22S35%22:%22E%22,%22S36%22:%22E%22,%22S37%22:%22E%22,%22S20%22:%22E%22,%22S21%22:%22E%22,%22S22%22:%22E%22,%22S23%22:%22E%22,%22S24%22:%22E%22,%22S25%22:%22E%22,%22S26%22:%22E%22,%22S27%22:%22E%22,%22S10%22:%22WP%22,%22S11%22:%22WP%22,%22S12%22:%2

((0, 3), (4, 7))


127.0.0.1 - - [06/Sep/2020 13:24:42] "GET /chess/pos/%7B%22pos%22:%7B%22S70%22:%22BR%22,%22S71%22:%22E%22,%22S72%22:%22BB%22,%22S73%22:%22BQ%22,%22S74%22:%22BK%22,%22S75%22:%22BB%22,%22S76%22:%22BN%22,%22S77%22:%22BR%22,%22S60%22:%22BP%22,%22S61%22:%22BP%22,%22S62%22:%22BP%22,%22S63%22:%22BP%22,%22S64%22:%22BP%22,%22S65%22:%22BP%22,%22S66%22:%22BP%22,%22S67%22:%22BP%22,%22S50%22:%22E%22,%22S51%22:%22E%22,%22S52%22:%22BN%22,%22S53%22:%22E%22,%22S54%22:%22E%22,%22S55%22:%22E%22,%22S56%22:%22E%22,%22S57%22:%22E%22,%22S40%22:%22E%22,%22S41%22:%22E%22,%22S42%22:%22E%22,%22S43%22:%22E%22,%22S44%22:%22E%22,%22S45%22:%22E%22,%22S46%22:%22E%22,%22S47%22:%22WQ%22,%22S30%22:%22E%22,%22S31%22:%22E%22,%22S32%22:%22E%22,%22S33%22:%22E%22,%22S34%22:%22WP%22,%22S35%22:%22E%22,%22S36%22:%22E%22,%22S37%22:%22E%22,%22S20%22:%22E%22,%22S21%22:%22E%22,%22S22%22:%22E%22,%22S23%22:%22E%22,%22S24%22:%22E%22,%22S25%22:%22E%22,%22S26%22:%22E%22,%22S27%22:%22E%22,%22S10%22:%22WP%22,%22S11%22:%22WP%22,%22S12%22:%

((0, 5), (3, 2))


127.0.0.1 - - [06/Sep/2020 13:25:19] "GET /chess/pos/%7B%22pos%22:%7B%22S70%22:%22BR%22,%22S71%22:%22E%22,%22S72%22:%22BB%22,%22S73%22:%22BQ%22,%22S74%22:%22BK%22,%22S75%22:%22BB%22,%22S76%22:%22E%22,%22S77%22:%22BR%22,%22S60%22:%22BP%22,%22S61%22:%22BP%22,%22S62%22:%22BP%22,%22S63%22:%22BP%22,%22S64%22:%22BP%22,%22S65%22:%22BP%22,%22S66%22:%22BP%22,%22S67%22:%22BP%22,%22S50%22:%22E%22,%22S51%22:%22E%22,%22S52%22:%22BN%22,%22S53%22:%22E%22,%22S54%22:%22E%22,%22S55%22:%22BN%22,%22S56%22:%22E%22,%22S57%22:%22E%22,%22S40%22:%22E%22,%22S41%22:%22E%22,%22S42%22:%22E%22,%22S43%22:%22E%22,%22S44%22:%22E%22,%22S45%22:%22E%22,%22S46%22:%22E%22,%22S47%22:%22WQ%22,%22S30%22:%22E%22,%22S31%22:%22E%22,%22S32%22:%22WB%22,%22S33%22:%22E%22,%22S34%22:%22WP%22,%22S35%22:%22E%22,%22S36%22:%22E%22,%22S37%22:%22E%22,%22S20%22:%22E%22,%22S21%22:%22E%22,%22S22%22:%22E%22,%22S23%22:%22E%22,%22S24%22:%22E%22,%22S25%22:%22E%22,%22S26%22:%22E%22,%22S27%22:%22E%22,%22S10%22:%22WP%22,%22S11%22:%22WP%22,%22S12%22: